### Matrix Sparsification

- Varying time granularities
- knn
- G-TGM
- Cluster D
- Cluster OD

In [100]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import Point
from sklearn.neighbors import NearestNeighbors
from math import radians, cos, sin, asin, sqrt,exp,pow
import random
import statistics

def distance_decay(distance, decay_constant, exponent):
    return exp(-decay_constant * pow(distance, exponent))

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [ ]:
#Params
decay_constant = 1
exponent = 0.09

In [3]:
#import OA data
wm_oas = gpd.read_file('data/west_midlands_OAs/west_midlands_OAs.shp')
wm_oas = wm_oas[wm_oas['LAD11CD'] == 'E08000026']
oa_info = pd.read_csv('data/oa_info.csv')
oa_info = oa_info.merge(wm_oas[['OA11CD']], left_on = 'oa_id', right_on = 'OA11CD', how = 'inner')
oaIndex = list(oa_info['oa_id'])
oaLatLon = oa_info[['oa_lon','oa_lat']].values

In [4]:
#import POI data
pois = pd.read_csv('data/POIs/pois.csv', index_col=0)

#Select local POIs
poisInRegion = []

for i,r in pois.iterrows():
    poiPoint = Point(tuple(list(r[['poi_lon','poi_lat']])))
    
    for i2, r2 in wm_oas.iterrows():
        if r2['geometry'].intersects(poiPoint):
            poisInRegion.append(r['poi_id'])

pois = pois[pois['poi_id'].isin(poisInRegion)]
pois = pois[pois['type']=='Vaccination Centre']

poiIndex = list(pois['poi_id'])
pois = pois[['poi_lat','poi_lon']].values

In [21]:
#Get OA and POI IDs

oas_in_study = list(set(list(trips['oa_id'])))
pois_in_study = list(set(list(trips['poi_id'])))

In [80]:
mean = 50
std_dev = 20
attractivnessDict = {}

for pid in poiIndex:
    random_value = random.normalvariate(mean, std_dev)    
    attractivnessDict[pid] = max(min(random_value, 100), 0)

In [104]:
#import trips data
trips = pd.read_csv('tempdata/tripscosts_otp.csv', index_col=0)

#Compute generalised access cost
trips['gac'] = (( 1.5 * (trips['duration'])) - (0.5 * trips['transit_time']) + ((trips['fare'] * 3600) / 6.7) + (10 * trips['transfers'])) / 60
trips['att'] = trips['poi_id'].map(attractivnessDict)

In [105]:
#GROUND TRUTH - Gravity Model

testDistsDecay = []
for i in np.array(trips['gac']):
    testDistsDecay.append(distance_decay(i, decay_constant, exponent))

trips['dist decay'] = testDistsDecay
trips['grav'] = trips['dist decay'] * trips['att']

gravity = trips.groupby('oa_id').sum()['grav']

print('Gravity model run')

Gravity model run


# 1. Varing Time Granularities

# 2. KNN

To do:
- Extend to further value of k
- Test on different time granularities
- Capture results in efficient manner
- Verify outputs

In [49]:
#for k in [5,10,15,20,25]:
k = 5
knn = NearestNeighbors(n_neighbors=k)
knn.fit(pois)

kResList = []
countOrigins = 0

nextOA = oas_in_study[0]
for nextOA in oas_in_study:
    o = oaLatLon[oaIndex.index(nextOA)]
    distances, indices = knn.kneighbors(o.reshape(1, -1))
    oCosts = []
    oTrips = 0
    #oTimes = 0
    for i in indices[0]:
        pid = poiIndex[i]
        trips_sample = trips[(trips['oa_id'] == nextOA) & (trips['poi_id'] == pid)]
        oCosts = oCosts + list(trips_sample['gac'])
        oTrips += len(trips_sample)
        #oTimes += tripSample['queryTime'].sum()

NearestNeighbors()

# 3. G-TGN

To do:
- Sort POI indexing
- Add query times
- Verify outputs

In [106]:
distMxList = []

for nextOA in oas_in_study:
    o = oaLatLon[oaIndex.index(nextOA)]
    p_iterator = 0
    for p in pois:
        pid = poiIndex[p_iterator]
        rowAppend = {}
        rowAppend['oa'] = nextOA
        rowAppend['poi'] = pid
        rowAppend['dist'] = haversine(o[0],o[1],p[1],p[0])
        distMxList.append(rowAppend)
        p_iterator += 1

distMx = pd.DataFrame(distMxList)
distMx['att'] = distMx['poi'].map(attractivnessDict)

distsDecay = []
for i in np.array(distMx['dist']):
    distsDecay.append(distance_decay(i, decay_constant, exponent))

distMx['decay'] = distsDecay
distMx['grav'] = distMx['decay'] * distMx['att']
distMx['gravN'] = (distMx['grav'] - distMx['grav'].min()) / (distMx['grav'].max() - distMx['grav'].min())
distMx = distMx.merge(trips.groupby(['oa_id','poi_id']).count()['time'].rename('tripCount'), left_on = ['oa','poi'],right_index = True)
distMx['tripsSample'] = distMx['tripCount'] * distMx['gravN']

In [107]:
resultsList = []
countOrigins = 0

print('Running Trip Gen Gravity')

for o in oas_in_study:
    countOrigins += 1
    oCosts = []
    oTrips = 0
    oTimes = 0
    for p in poiIndex:
        numSample = int(distMx[(distMx['oa'] == o) & (distMx['poi'] == p)]['tripsSample'].values[0])
        tripSample = trips[(trips['oa_id'] == o) & (trips['poi_id'] == p)].sample(numSample)
        oCosts = oCosts + list(tripSample['gac'])
        oTrips += numSample
        #oTimes += tripSample['queryTime'].sum()

    rowAppend = {}
    rowAppend['OA'] = o
    rowAppend['GTG'] = statistics.mean(oCosts)
    rowAppend['GTG_Trips'] = oTrips
    rowAppend['GTG_Times'] = oTimes
    resultsList.append(rowAppend)

tripGenGrav = pd.DataFrame(resultsList)
zoneLevelResults = tripGenGrav.merge(gravity, left_on = 'OA', right_index = True)

print('Fnished trip gen grav')

Running Trip Gen Gravity
Fnished trip gen grav


# 4. Spatial clustering of destinations

To do:

- k-means clustering on POIs
- Get POI closest to centroid in each cluster
- Measure access to this POI for all
- Run a gravity model

Unknowns:
- How to handle attractiveness (e.g., attractiveness weighted clustering?)

# 5. OD Clustering using flow clustering